# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [ ]:
# from openai import OpenAI
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')


client = OpenAI(api_key=OPENAI_API_KEY)


In [63]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [64]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [65]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Forsooth, the chicken did cross yon road to reach the other side to avoid being accused of fowl play! Ha ha!


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, I don't have that information, could you tell me what is your name?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [9]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'bb0a1136-5ab5-4156-884c-43e1ea7c495a': {'version…

In [52]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "pizza": {
        "type": "pepperoni",
        "size": "large",
        "price": 12.95
    },
    "toppings": [],
    "drinks": [
        {
            "type": "sprite",
            "size": "regular",
            "price": 2.00
        }
    ],
    "sides": [
        {
            "type": "fries",
            "size": "regular",
            "price": 4.50
        }
    ],
    "total_price": 20.45
}
```


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [57]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an AI assistant for a restaurant. \
You first greet the customer, then collect the order, \
and then ask if it's a pickup or delivery. \
You summarize the order and check if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally, you collect the payment.\
Make sure to clarify all options, extras, and sizes.\
You respond in a short, very conversational friendly style. \
The menu includes: \
Burger - 9.99 (Large), 7.50 (Medium), 6.00 (Small) \
Chicken Nuggets - 8.50 (Large), 6.50 (Medium), 5.00 (Small) \
French Fries - 3.50 (Large), 2.50 (Small) \
Drinks: \
Pepsi - 2.00 (Large), 1.50 (Medium), 1.00 (Small) \
Lemonade - 3.00 (Large), 2.00 (Medium), 1.50 (Small) \
"""} ]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter your order here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


Column
    [0] TextInput(placeholder='Enter your order here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [58]:
context = [ {'role':'system', 'content':"""
You are OrderBot, a highly professional AI assistant for a fine dining restaurant. \
You communicate formally and provide detailed information about each menu item. \
Your responses are polite, structured, and informative.\
The menu includes: \
 Gourmet Steak - 29.99 (Large), 24.50 (Medium), 19.00 (Small) \
 Truffle Pasta - 22.50 (Large), 18.50 (Medium), 15.00 (Small) \
 Caesar Salad - 12.50 \
 Drinks: \
Red Wine - 15.00 per glass \
Sparkling Water - 5.00 \
"""} ]

dashboard


Column
    [0] TextInput(placeholder='Enter your order here…', value_input='How much is the total?')
    [1] Row
        [0] Button(clicks=3, name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [59]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an AI assistant for a restaurant. \
You offer combo meals with discounted prices. \
Your menu includes: \
 Burger - 9.99 (Large), 7.50 (Medium), 6.00 (Small) \
 Chicken Nuggets - 8.50 (Large), 6.50 (Medium), 5.00 (Small) \
 French Fries - 3.50, 2.50 \
 Drinks: \
Pepsi - 2.00, 1.50, 1.00 \
Lemonade - 3.00, 2.00, 1.50 \
 Combo Meal: Burger + Fries + Drink = 12.99 \
"""} ]

dashboard.show()


Column
    [0] TextInput(placeholder='Enter your order here…', value_input='Hello, I would l...)
    [1] Row
        [0] Button(clicks=4, name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [61]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an AI assistant for a restaurant. \
You must always ask the customer how they would like to pay (cash, credit card, or online payment) before finalizing the order.\
The menu includes: \
 Pizza - 10.99 (Large), 8.50 (Medium), 6.50 (Small) \
 Burger - 9.99 (Large), 7.50 (Medium), 6.00 (Small) \
 French Fries - 3.50, 2.50 \
 Drinks: \
Coke - 2.00, 1.50, 1.00 \
Lemonade - 3.00, 2.00, 1.50 \
"""} ]

dashboard

Column
    [0] TextInput(placeholder='Enter your order here…', value_input='thatsit')
    [1] Row
        [0] Button(clicks=9, name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

# 📄 OrderBot Experiment Report

### 1️⃣ Modifications Implemented:
During this exercise, I tested four different modifications to the OrderBot to enhance user experience and make the assistant more interactive. The modifications were as follows:

Adding a new menu for a fast-food restaurant – Updated OrderBot to take burger and nuggets orders instead of pizza.
Making the assistant speak in a formal tone for a fine dining restaurant – Introduced an upscale menu and structured, professional responses.
Adding a combo meal with a discount – Allowed users to order a meal that includes a burger, fries, and a drink at a reduced price.
Forcing the assistant to ask for a payment method before finalizing the order – Ensured OrderBot always asks about payment preferences before confirming the order.

### 2️⃣ Results and Observations:
First Modification (Fast-Food Menu): Successfully displayed the new menu and allowed OrderBot to process different order requests smoothly.
Second Modification (Formal Style): Worked well, making OrderBot more precise and structured, but it felt less friendly, which might make interactions less engaging for some users.
Third Modification (Combo Meal): The assistant correctly recognized the combo meal and applied the discounted price, but sometimes asked unnecessary follow-up questions about portion sizes, which could be optimized.
Fourth Modification (Payment Method Request): Successfully ensured that OrderBot asked for payment before finalizing the order. However, sometimes it would ask for payment details at the end instead of before confirmation, which could be improved by refining the prompt structure.

### 3️⃣ Key Learnings:
Even small prompt modifications significantly impact OrderBot's responses, highlighting the importance of clear and well-structured instructions.
Changing the conversational tone affects user experience – The formal tone was more precise but less engaging, while the casual tone made interactions more enjoyable.
Adding features like combo meals and improving order finalization steps makes the assistant more functional, but further testing is required to ensure a smooth experience.


### 📌 Conclusion:
Customizing OrderBot and improving interaction style and menu structure significantly enhanced the user experience. I learned that clear prompts and instructions are crucial for obtaining accurate responses, and continuous iteration is necessary to refine the assistant’s behavior.